In [ ]:
from datasets import load_from_disk
import matplotlib.pyplot as plt

In [ ]:
dataset = load_from_disk('../data/translation_eval_results/translate_evaluation_results/')
df = dataset.to_pandas()

In [ ]:
idx = 110

In [ ]:
for m, c1, c2, w in zip(df.loc[idx].messages, df.loc[idx].translated_reponse_m4t, df.loc[idx].translated_reponse_gpt, df.loc[idx].winners):
    print(m['content'])
    print('\n')
    print(f'm4t: {c1}')
    print(f'cgpt: {c2}')
    print(f'winner: {w}')
    print('\n')

In [ ]:
results = {
    'translated_reponse_m4t': 0,
    'translated_reponse_gpt': 0
}

for elem in df.winners.to_list():
    for e in elem:
        results[e] += 1

In [ ]:
results

In [ ]:
dataset = load_from_disk('../data/tmp/2023122114/translation_eval/')
df = dataset.to_pandas()

In [ ]:
idx = 1

In [ ]:
for idx in range(df.shape[0]):
    for m, c1, c2, c3 in zip(df.loc[idx].messages, df.loc[idx].translated_reponse_m4t, df.loc[idx].translated_reponse_gpt, df.loc[idx].translated_reponse_gemini):
        print(m['content'])
        print('\n')
        print(f'm4t: {c1}')
        print(f'cgpt: {c2}')
        print(f'gemini: {c3}')
        print('\n')
    print('\n\n\n\n')

In [ ]:
ds = load_from_disk('../data/synthetic_data/20231227-2038/general_mcq/')
print(ds.num_rows)

In [ ]:
ds.column_names

In [ ]:
grade = ds.select_columns('GRADE').to_list()
grade = [c['GRADE'] for c in grade]

plt.hist(grade)
plt.show()

In [ ]:
sub = ds.select_columns('SUBJECT').to_list()
sub = [c['SUBJECT'] for c in sub]

plt.hist(sub)
plt.show()

## notes

I looked at 4 things to evaluate the synthetic data:
1. Should follow instruction of generating question in either hindi or romanized hindi
2. Answer should be correct
3. Question should make logical sense
4. No gibberish in the question/answer

gpt3.5 performs very poor in generating synthetic data in hindi. Target can be either the answer text or option. Sometimes doesn't follow instructions to write in Hindi. Answer is not always correct and errors are high in chemistry and biology and 10th and 12th standard.


gpt4 performs much better thant gpt3.5, but for biology and chemistry, the performance is not that great. So Ill reduce biology and chemistry topics and for maths and physics, will keep less 10th and 12th standard questions.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import openai
import random
import json
from textwrap import dedent

In [ ]:
client = openai.OpenAI()

In [ ]:
system_prompt = {
    'role': 'system',
    'content': dedent('''
        You are an helpful assistant who verifies questions given in JSON format.
        For every question, verify these things and reply:
            1. Does the QUESTION make logical sense? Reply in one word, TRUE or FALSE
            2. Is there any gibberish present in the QUESTION? Reply in one word, TRUE or FALSE
            3. Is the answer in the TARGET field correct with respect to the question? Reply in one word, TRUE or FALSE
            4. What language is the QUESTION in (Devnagri Hindi, Romanized Hindi, English or other)? Reply with the language name only

        Always reply in JSON
    ''').strip()
}

user_prompt = {
    'role': 'user',
    'content': None
}

In [ ]:
sample = ds.filter(lambda x: x['SUBJECT'] == 'Biology')
sample = sample.select(
    random.sample(range(sample.num_rows), 10)
)

print(sample.num_rows)

In [ ]:
ops = []

for elem in iter(sample):
    user_prompt.update({
        'content': str(elem)
    })

    completions = client.chat.completions.create(
        model='gpt-4-1106-preview',
        response_format={'type': 'json_object'},
        messages=[
            system_prompt,
            user_prompt
        ],
        max_tokens=1000
    )
    try:
        op = json.loads(completions.choices[0].message.content)
        op.update({'SUBJECT': elem['SUBJECT'], 'GRADE': elem['GRADE']})
        print(op)
        ops.append(op)
    except:
        raise ValueError('Value returned by the model is not valid JSON')

In [ ]:
mental_maths = ops

In [ ]:
chemistry_results = ops

In [ ]:
maths_results = ops

In [ ]:
pyhsics_results = ops

In [ ]:
biology_results = ops

In [ ]:
arr = maths_results
illogical, gibberish, incorrect = 0, 0, 0
grade = []

for idx, e in enumerate(arr):
    flag = 0
    if e['1'] == 'FALSE':
        illogical += 1
        flag = 1
    if e['2'] == 'TRUE':
        gibberish += 1
        flag = 1
    if e['3'] == 'FALSE':
        incorrect += 1
        flag = 1

    if flag:
        grade.append(e['GRADE'])
        print(e)
        print(sample[idx])

print(dedent(f"""
Illogical: {illogical}
Gibberish: {gibberish}
Incorrect: {incorrect}
Total: {len(arr)}
"""))

plt.hist(grade)
plt.show()

In [ ]:
for elem in iter(sample):
    print(elem)
    print('\n')